In [1]:
import pandas as pd
import numpy as np
from nlpUtils import aardvark as aa 
from numpy import random as rand
from sklearn.metrics import f1_score, roc_curve, auc

import seaborn as sns
import matplotlib.pyplot as plt

import torch
from transformers import logging
logging.set_verbosity_error()  # Turns off the warning when you load the model without training
from transformers import BertForSequenceClassification
from transformers import BertTokenizerFast
from transformers import BertTokenizer

from sklearn.metrics import accuracy_score, recall_score, precision_score
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

In [2]:
X_train = ["this sucks", "the long dark nightime of the soul", "oh happy day", "this is just sunshine and rainbows", "piss on you and the horse you rode in on", "this is so"]
X_val = ["my kingdom for a focking horse", "this is not what i had hoped for", "my heart sings with joy", "i am not happy", "i am happy", "the sky is blue"]
Y_train = [0, 0, 1, 1, 0, 2]
Y_val = [0, 0, 1, 0, 1, 2]

In [3]:
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name)
#tokenizer = BertTokenizerFast.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name, num_labels=3)

In [4]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=40)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=40)

In [5]:
## THANKS TO: https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b
# Create torch dataset 
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

train_dataset = Dataset(X_train_tokenized, Y_train)
val_dataset = Dataset(X_val_tokenized, Y_val)


In [6]:
## THANKS TO: https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
# model.resize_token_embeddings(len(tokenizer))

# Train pre-trained model
trainer.train()

c:\Users\rnocker\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.